In [1]:
pip install openai

In [2]:
import openai
import time

# Initialize the OpenAI API with your key
openai.api_key = 'Give your API Key'

class LLMWrapper:
    MAX_TOKENS = 2048
    RATE_LIMIT_SLEEP_DURATION = 10

    def __init__(self):
        self.conversation_history = []
        self.current_prompt = {}

    def call_llm_api(self, text):
        if len(text) > LLMWrapper.MAX_TOKENS:
            raise ValueError("Token Limit Exceeded!")

        messages = [{"role": "system", "content": "You are a helpful assistant."}]

        # Add history to the messages
        for entry in self.conversation_history:
            messages.append({"role": "user", "content": entry['user']})
            messages.append({"role": "assistant", "content": entry['llm']})

        # Add current user input last
        messages.append({"role": "user", "content": text})

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=500
        )

        return response.choices[0].message['content'].strip()

    def format_prompt(self):
        return "; ".join([f"{k}: {v}" for k, v in self.current_prompt.items()])

    def update_prompt_variables(self, new_variables):
        self.current_prompt.update(new_variables)

    def add_to_history(self, user_input, llm_response):
        self.conversation_history.append({
            "user": user_input,
            "llm": llm_response,
            "prompt": self.format_prompt()
        })

    def get_formatted_conversation(self):
        return "\n".join([f"User: {entry['user']}\nLLM: {entry['llm']}\nPrompt: {entry['prompt']}" for entry in self.conversation_history])

    def handle_rate_limit_error(self):
        print("Rate Limit Error! Waiting...")
        time.sleep(LLMWrapper.RATE_LIMIT_SLEEP_DURATION)

    def handle_token_limit_error(self):
        while len(self.get_formatted_conversation() + self.format_prompt()) > LLMWrapper.MAX_TOKENS:
            self.conversation_history.pop(0)

    def send_to_llm(self, user_input, **prompt_variables):
        self.update_prompt_variables(prompt_variables)

        while True:
            try:
                self.handle_token_limit_error()
                llm_response = self.call_llm_api(user_input)
                self.add_to_history(user_input, llm_response)
                return llm_response
            except ValueError as e:
                if "Token Limit Exceeded" in str(e):
                    self.handle_token_limit_error()
                else:
                    raise e
            except Exception as e:
                if "rate limit" in str(e).lower():
                    self.handle_rate_limit_error()
                else:
                    raise e

# Usage
wrapper = LLMWrapper()

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    prompt_variables = {}

    response = wrapper.send_to_llm(user_input, **prompt_variables)
    print(f"LLM: {response}\n")

You: Hello
LLM: Hi there! How can I assist you today?

You: Let X = 1
LLM: Great! X is now equal to 1. What would you like to do with this value of X?

You: Now Consider Y = X + 10
LLM: Alright, if X = 1, then Y would be equal to 11 (since Y = X + 10). Is there anything else you would like me to help you with regarding these variables?

You: Z = Y + X - 11
LLM: Got it! If Y = 11 and X = 1, then Z would be equal to 11 + 1 - 11, which simplifies to 1. Is there anything else you would like me to assist you with?

You: Value of Z = ?
LLM: Based on the given values of X=1 and Y=11, the value of Z would be 1.

You: exit
